<a href="https://colab.research.google.com/github/tetsufmbio/iremolog/blob/main/notebooks/pipeline_remolog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IRemolog: identifying remote homologs using structural alignment data

## Quick start

1. Press "Runtime" --> "Run all".
2. In the next cell (*Upload protein structure*), it will appear the bottom "Choose file". Click on it and choose one or more pdb file to be uploaded and analyzed by the pipeline*.
3. After the running, it will download a file named "remolog_final_result.tab". See its description in the end of this notebook.
4. Running time of this pipeline in Colab take ~10 min when using foldseek for screening**.

\* If you have only the amino acid sequence, you can predict its structure using the [AlphaFold Colab notebook](https://github.com/sokrypton/ColabFold)

\** If no remote homolog was found, try to increase the parameter "n" or change the screening method.


In [ ]:
%cd /content
! if [ -d input ]; then \
    rm -rf input; \
  fi
! mkdir input


%cd input

from google.colab import files

uploaded = files.upload()
#@markdown  By running this cell, a bottom "Choose files" will appear. Click on it and choose the pdb file to be analyzed (you may upload multiple files).

%cd /content


## Setting parameters

In [ ]:
screening = "foldseek" #@param ["foldseek","tmalign", "fatcat"]
#@markdown  - software to be used to retrieve structurely similar proteins in SCOPe database.

n = 20 #@param {type:"integer"}
#@markdown  - analyze n most similar protein structures.

database = "scope40" #@param ["scope40", "scope95"]
#@markdown  - protein structure database to be used.

jobName = "remolog_final_result" #@param {type: "string"}
#@markdown  - prefix for the final output name.


## Installing dependencies and downloading database

In [ ]:
# setup environment variables
import os
os.environ['FATCAT'] = '/content/programs/FATCAT-dist'
os.environ['PATH'] += ':/content/programs/FATCAT-dist/FATCATMain:/content/bin'
os.environ['HEADN'] = str(n)
os.environ['SCREEN'] = screening
os.environ['DATABASE'] = database
if (database == "scope40"):
  os.environ['ANNOT'] = "/content/programs/iremolog/data/dir.cla.scope.2.08-stable_filtered40.txt"
elif (database == "scope95"):
  os.environ['ANNOT'] = "/content/programs/iremolog/data/dir.cla.scope.2.08-stable_filtered95.txt"


In [ ]:
%cd /content/
! if [ ! -d bin ]; then mkdir bin; fi
! if [ ! -d programs ]; then mkdir programs; fi
! if [ ! -d view ]; then mkdir view; fi
! if [ ! -d foldseek_data ]; then mkdir foldseek_data; fi

In [ ]:
# download and install TMalign
%cd /content/bin
! if [ ! -e TMalign ]; then \
    wget "https://zhanggroup.org/TM-align/TMalign.cpp"; \
    g++ -static -O3 -ffast-math -lm -o TMalign TMalign.cpp; \
  fi


In [ ]:
# download and install FATCAT
%cd /content/programs
! if [ ! -d FATCAT-dist ]; then \
    git clone https://github.com/GodzikLab/FATCAT-dist.git; \
    cd FATCAT-dist/; ./Install; \
  fi


In [ ]:
# download and install lovoalign
%cd /content/bin
! if [ ! -e lovoalign ]; then \
    wget "https://github.com/m3g/lovoalign/archive/refs/tags/22.0.0.tar.gz"; \
    tar -xzf 22.0.0.tar.gz; cd lovoalign-22.0.0/src; make; cp ../bin/lovoalign /content/bin; \
  fi

In [ ]:
# download some scripts and model
%cd /content/programs
! if [ ! -d iremolog ]; then \
    git clone https://github.com/tetsufmbio/iremolog.git; \
  fi

In [ ]:
%cd /content/programs
! if [ ! -e /content/bin/foldseek ]; then \
    while true; do \
      wget -T 15 -c https://mmseqs.com/foldseek/foldseek-linux-sse2.tar.gz && break ; \
    done \
  fi
! tar xvzf foldseek-linux-sse2.tar.gz; cp foldseek/bin/foldseek /content/bin;

In [ ]:
# download and format scope database
%cd /content
! if [ ! -d database ]; then \
    mkdir database; \
  fi;
%cd /content/database
! if [ ! -d $DATABASE ]; then \
      mkdir $DATABASE; \
  fi;
%cd /content/database/$database
! if [ ! -f ../list_$DATABASE.tab ]; then \
    if [ $DATABASE = "scope40" ]; then \
      wget https://scop.berkeley.edu/downloads/pdbstyle/pdbstyle-sel-gs-bib-40-2.08.tgz; \
      tar -zxf pdbstyle-sel-gs-bib-40-2.08.tgz; mv pdbstyle-2.08/*/*.ent . ; rm -rf pdbstyle*; \
    elif [ $DATABASE = "scope95" ]; then \
      wget https://scop.berkeley.edu/downloads/pdbstyle/pdbstyle-sel-gs-bib-95-2.08.tgz; \
      tar -zxf pdbstyle-sel-gs-bib-95-2.08.tgz; mv pdbstyle-2.08/*/*.ent . ; rm -rf pdbstyle*; \
    fi; \
    ls *.ent > ../list_$DATABASE.tab; for i in *.ent; do mv $i $i.pdb; done; \
  fi;

In [ ]:
# creating foldseek database
%cd /content/foldseek_data/
! if [ ! -f fs_$database ]; then \
  foldseek createdb /content/database/$database fs_$database ; \
fi

## Running protein structure alignment

In [ ]:
%cd /content
! if [ -d result ]; then rm -rf result ; fi
! mkdir result;
! mkdir result/screening;

In [ ]:
# screening for similar proteins using foldseek
%cd /content/input

! if [ $SCREEN = "foldseek" ]; then \
    for f in *; do \
      foldseek easy-search $f /content/foldseek_data/fs_$DATABASE /content/result/screening/tmp.tab.fmt /content/tmpFolder --max-seqs $HEADN -e inf; \
      cut -f 1,2 /content/result/screening/tmp.tab.fmt | sort | uniq | perl -ne '@a = split(/\./, $_); print join(".", @a[0 .. $#a-2])."\n";' > /content/result/screening/$f.tab.fmt; \
      rm /content/result/screening/tmp.tab.fmt; \
    done; \
  fi

In [ ]:
# screening for similar proteins using FATCAT

%cd /content/input
! if [ $SCREEN = "fatcat" ]; then \
    for f in *; do \
      FATCATSearch.pl $f /content/database/list_$DATABASE.tab -b -i1 /content/input -i2 /content/database/$DATABASE | \
      sort -k11nr | \
      head -n $HEADN | \
      perl /content/programs/remolog/scripts/format_result_FATCAT.pl - /content/programs/remolog/data/maxScore_fatcat.tab > /content/result/screening/$f.tab.fmt; \
    done; \
    cat /content/result/screening/*.fmt > /content/result/fatcat_formatted.tab; \
  fi

In [ ]:
# screening for similar proteins using tmalign

%cd /content/input
! if [ $SCREEN = "tmalign" ]; then \
  for f in *; do \
    if [ -f /content/result/screening/${f}.tab ]; then \
      rm /content/result/screening/${f}.tab; \
    fi; \
    for l in $(cat /content/database/list_$DATABASE.tab); \
      do TMalign /content/input/$f /content/database/$DATABASE/${l}.pdb | perl /content/programs/iremolog/scripts/parser_TMalign.pl - >> /content/result/screening/${f}.tab ; \
      done; \
    sort -k3nr /content/result/screening/${f}.tab | grep ${f} | head -n $HEADN > /content/result/screening/${f}.tab.fmt; \
    done; \
  cat /content/result/screening/*.fmt > /content/result/tmalign_formatted.tab; \
fi

In [ ]:
%cd /content/input
! if [ $SCREEN != "fatcat" ]; then \
  if [ -f /content/result/fatcat_formatted.tab ]; then \
    rm /content/result/fatcat_formatted.tab; \
  fi; \
  for f in *; do \
    for l in $(cut -f2 /content/result/screening/$f.tab.fmt); do \
      FATCAT -p1 $f -p2 $l.ent.pdb -i1 /content/input -i2 /content/database/$DATABASE -b | \
      perl /content/programs/iremolog/scripts/format_result_FATCAT.pl - /content/programs/iremolog/data/maxScore_fatcat.tab >>  /content/result/fatcat_formatted.tab ; \
    done; \
  done; \
  fi

In [ ]:
%cd /content/input
! if [ $SCREEN != "tmalign" ]; then \
    if [ -f /content/result/tmalign_formatted.tab ]; then \
      rm /content/result/tmalign_formatted.tab; \
    fi; \
    for f in *; do \
      for l in $(cut -f2 /content/result/screening/$f.tab.fmt); do \
        TMalign /content/input/$f /content/database/$DATABASE/$l.ent.pdb | perl /content/programs/iremolog/scripts/parser_TMalign.pl - >> /content/result/tmalign_formatted.tab ; \
      done; \
    done;\
  fi

In [ ]:
%cd /content/input
! if [ $SCREEN != "lovoalign" ]; then \
    if [ -f /content/result/lovoalign_formatted.tab ]; then \
      rm /content/result/lovoalign_formatted.tab; \
    fi; \
    for f in *; do \
      for l in $(cut -f2 /content/result/screening/$f.tab.fmt); do \
        lovoalign -p1 /content/input/$f -p2 /content/database/$DATABASE/$l.ent.pdb | perl /content/programs/iremolog/scripts/parser_lovoalign.pl - >> /content/result/lovoalign_formatted.tab; \
      done; \
    done; \
  fi

In [ ]:
%cd /content/result
! perl /content/programs/iremolog/scripts/join_table.pl /content/result/fatcat_formatted.tab /content/result/tmalign_formatted.tab | \
  perl /content/programs/iremolog/scripts/join_table.pl - /content/result/lovoalign_formatted.tab | \
  perl /content/programs/iremolog/scripts/add_scope_class.pl - $ANNOT > result.tab

## Making prediction and writing the results

In [ ]:
# create model
%cd /content/programs/iremolog/scripts
!if [ ! -f model.joblib ]; then python3 /content/programs/iremolog/scripts/create_model.py ../data/data.csv; fi

In [ ]:
# make prediction
%cd /content/programs/iremolog/scripts
!python3 classify.py /content/result/result.tab model.joblib

In [ ]:
# summarize result
%cd /content/programs/iremolog/scripts
!perl add_desc.pl result.tab ../data/dir.des.scope.2.08-stable_sf.txt $ANNOT > result_summary.tab

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("/content/programs/iremolog/scripts/result_summary.tab", sep="\t")


In [ ]:
data = data.sort_values(by=["query", "pred_proba"], ascending=[True, False])

In [ ]:
# write and download result file
from google.colab import files

data.to_csv("/content/result/"+jobName+".tab", index=False)
files.download("/content/result/"+jobName+".tab")

In [ ]:
# function to display table
from google.colab import data_table
data_table.enable_dataframe_formatter()

def hyperlink(path):
	
    # returns the substring of a path

    pathList = path.split("=")
    f_url = pathList[len(pathList)-1]
    path="https://scop.berkeley.edu/search/?ver=2.08&key="+f_url
    #print(f_url)
    
    # convert the path into clickable link
    return '<a target="_blank" href="{}">{}</a>'.format(path, f_url)


In [ ]:
# functions to display 3d alignment
! pip install py3Dmol
import py3Dmol
import glob
import matplotlib.pyplot as plt 

In [ ]:
import ipywidgets as widgets

def print_query(subject):
    print(subject)

def select_subject(query):
    subject_picker.options = list(data[data["query"] == query].subject)

query_list = data.loc[:,"query"].unique()
query_picker = widgets.Dropdown(options=query_list, value=query_list[0])
subject_list = list(data[data["query"] == query_list[0]].subject)
subject_picker = widgets.Dropdown(options=subject_list, value=subject_list[0])
j = widgets.interactive(print_query, subject=subject_picker)
i = widgets.interactive(select_subject, query=query_picker)

#button = widgets.Button(description="Submit")
#output = widgets.Output()
#display(button)

def display_str():
  selected_subject = data[(data["subject"] == subject_picker.value) & (data["query"] == query_picker.value)]
  os.chdir("/content/view")
  os.system("rm *")
  os.system("FATCAT -p1 "+query_picker.value+" -p2 "+subject_picker.value+".ent.pdb -i1 /content/input -i2 /content/database/$DATABASE -t")

  with open("/content/view/tmp.opt.twist.pdb") as ifile:
      system = "".join([x for x in ifile])
      
  view = py3Dmol.view(width=400, height=300)
  view.addModelsAsFrames(system)

  view.setStyle({'chain':'A'}, {'cartoon':{'color':'blue'}})
  view.setStyle({'chain':'B'}, {'cartoon':{'color':'yellow'}})
  view.zoomTo()
  view.show()


  display(pd.DataFrame(selected_subject))


#button.on_click(on_button_clicked)



## Display table 

In [ ]:
data2 = data.style.format({'subject': hyperlink, 'sf': hyperlink})
data2

In [ ]:
data_table.disable_dataframe_formatter()

Description of the columns
- query: Query name
- subject: Subject name

Lovoalign
- lovo_finalScore: Final score;
- lovo_coverage: Alignment coverage;
- lovo_rmsd: RMSD;
- lovo_gaps: # of gaps;
- lovo_relCov: proportion of the coverage and subject length
- lovo_relGaps: propotion of gaps to coverage
- lovo_finalScoreNorm: Normalized score;

TM-align
- tm_AliLen: Alignment length;
- tm_RMSD: RMSD;
- tm_n_ident/n_aln: proportion of # identical atom and aligned length;
- tm_TM-score (chain 2): TM-score normalized by subject;
- tm_d0 (chain 2): scale factor used to calculate TM-score; 
- tm_cov: coverage of the alignment (subject)

FATCAT
- fatcat_subject-len: subject length;
- fatcat_Twists: # of twists;
- fatcat_ini-len: Initial alignment length;
- fatcat_ini-rmsd: Initial RMSD;
- fatcat_opt-equ: # of equivalent positions in the alignment;
- fatcat_opt-rmsd: RMSD of aligned Cα atoms of the input structures with structural rearragement;
- fatcat_chain-rmsd: RMSD of aligned Cα atoms of the input structures without structural rearragement;
- fatcat_Score: Alignment score
- fatcat_align-len: Alignment length 
- fatcat_Gaps: # of gaps in the alignment
- fatcat_rel_score: proportion of the alignment score and maximum score
- fatcat_rel_align: proportion of the # of aligned position with subject length

Prediction
- pred: prediction (0: not remote homolog; 1: remote homolog)
- pred_proba: prediction probability

SCOPe annotation
- cl: subject SCOPe class
- cf: subject SCOPe fold
- sf: subject SCOPe superfamily

## Display protein structure alignment

In [ ]:
print("Choose a query")
display(query_picker)
print("Choose the Subject")
display(subject_picker)

In [ ]:
# Choose a query and a subject in the cell above and run this cell to display
# the structure alignement performed by FATCAT.
# chain in blue: Query
# chain in yellow: Subject
display_str()